# An Introduction to Decorators in Python

### Python@UH #2 

#### Mahdi Belcaid

#### mahdi@hawaii.edu

### Use case 1: Logging

* Consider a function `do_some_work()`. How can you extend it?

  * Ex. Log the execution of `do_some_work()` when the variable DEBUG is True

```python 
If debug == True:
    print(“I am calling `do_som_work`”)
    do_some_work()
```
* The above required adding an `if` statement before each call to do_some_work()



### Use case 2: Authentication


* Ascertain that `do_some_work` is only executed when a user is logged in

  * Require implementing a solution similar to the one in the previous example

```python
if user.is_logged:
    do_some_work()
else:        
    raise Exception(“User not logged cannot invoke do_some_work()”)
```


### Use case 3: Profiling

* Compute the time it takes to run of the function

```python
start_time = time.time()
do_some_work()
Total_time += time.time() - start_time
```

### Trivial Solution
     
* Why not just create another function that calls do work and injects the additional functionality
```
def check_logged_and_do_some_work():
    If user.logged == True:
        do_some_work
    else:
        raise Exception(“User not logged cannot invoke do_some_work()”)
```

* While this solution works, it does not scale.
    * Image creating functions for `check_logged_and_access_db`, `check_logged_and_store_results`, `check_logged_and_simulate_data`, etc...
    
* This shortcoming also applies to use cases 2 and 3.

### The Decorator Design Pattern

* Problem: we need to extend a function by typically executing code before and/or after it
  * We don't want to modify the original function
  
* This is a common problem that warrants its software engineering design pattern 
  * The pattern is called decorator

```
Decorator is a structural design pattern that lets you attach new behaviors to objects by placing these objects inside special wrapper objects that contain the behaviors. https://refactoring.guru/design-patterns/decorator
```


### Decorators in Python

* Function X that takes function Y returns a function Z  
  * Function Z wraps and extends function Y


### Functions in Python

Pythons function are first-class objects
E.g.:
    * Functions can be stored, retrieved, and called from a variable or a  collection
    * Function can be passed as arguments to other functions
    * Functions can be defined in any scope, including inside other functions




In [5]:
def return_greeting(name="John"):
    return f"Hello, {name}"

return_greeting(name="Mahdi")

'Hello, Mahdi'

In [6]:
build_greeting = return_greeting
build_greeting("Mahdi")

'Hello, Mahdi'

In [183]:
def call_some_function(func):
    return func()
    
call_some_function(build_greeting)

'Hello, John'

### Difference Between a Function and a Function Call 

* `return_greeting` is a function.

* `return_greeting()` is a funtion call, i.e., the string it returns

`call_some_function(func)` takes a function as input not a function call, which returns a string
```python
    call_some_function(return_greeting)           # Correct
    call_some_function(return_greeting("Mahdi"))  # Incorrect 
```

### Function Inside Other functions
```python
def parent():
    def nested_child():
        print("I am in the the nested child function")
    nested_child()
```


    
* The scope of the nested function is local to the parent function
  * We can only call the nested function from within the parent function
  


### Function Inside Other functions -- cont'd

* Alternatively, we can return the nested function and reuse it later
  * Return the *function* not the *function call*

```python
def parent():
    def nested_child():
        print("I am in the the nested child function")
    return nested_child
    
the_nested_func = parent()
the_nested_func()
```

In [2]:
def parent():
    def nested_child():
        print("I am in the the nested child function")
    nested_child()
parent()

I am in the the nested child function


In [185]:
def parent():
    def nested_child():
        print("I am in the the nested child function")
    return nested_child
    
the_nested_func = parent()
the_nested_func()

I am in the the nested child function


In [186]:
the_nested_func

<function __main__.parent.<locals>.nested_child()>

### Decorator
  * Recall that a decorator is simply: 
  ```Function X that takes function Y returns a function Z that wraps and extends Y`` 

```python 
def X(Y) :
    Z():
        # Do Something before Y()    
        Y()
        # Do something after Y()
    return Z
```

* We call the functions
  * X: decorator
  * Y: Decorated
  * Z: the wrapper

In [3]:
def X(Y):
    def Z():
        print("About to run Y")
        Y()
        print("Done running Y ")
    return Z


In [4]:
def Y():
    print ("doing some important work")

W = X(Y)

W()

About to run Y
doing some important work
Done running Y 


In [190]:
W

<function __main__.X.<locals>.Z()>

### Decorators: Syntactic Sugar

* We know everything about decorator in Python
  * The rest is syntactic sugar

* Python provides the `@` operator to simplify the decoration process

```python
@X
def Y():
    print("Some important work")
```    



In [5]:
@X
def Y():
    print("Some important work")
Y()

About to run Y
Some important work
Done running Y 


In [6]:
@X
def M():
    print("This is also important work")
M()

About to run Y
This is also important work
Done running Y 


#### Decorators Code Convention - 1

* We usually name the decorators and wrappers in a way that makes code easier to read.

```python

def X(fun):
    def wrapper_X():
        # Do somehting before, if needed
        fun()
        # Do somehting after, if needed
    return wrapper_X

@X
def hello():
    pass
```

* `hello` is now an alias for `wrapper_x`

In [10]:
def X(fun):
    def wrapper_X():
        print("About to run fun")
        fun()
    return wrapper_X

@X
def return_greeting():
    return "Hello!"


return_greeting()

About to run fun


#### Decorators Code Convention - 2

* We need to return the value returned by the decorated function (X)

```python

def X(fun):
    def wrapper_X():
        # Do somehting before, if needed
        val = fun()
        # Do somehting after, if needed
        return val
    return wrapper_X

@X
def return_greeting():
    return f"Hello!"
```
* if `return_greeting()` returns a value, then its alias, `wrapper_x`, will also need to return a value

In [11]:

def log_activity(fun):
    def wrapper_log_activity():
        print("Before execution of fun")
        val = fun()
        print("After execution of fun")
        return val
    return wrapper_log_activity


@log_activity
def return_greeting(name="John"):
    return f"Hello, {name}"

x = return_greeting()
print(f"\n\nThe returned value is: '{x}'.")

Before execution of fun
After execution of fun


The returned value is: 'Hello, John'.


### Passing Arguments to Decorators

* What does the following return?

```python
def log_activity(fun):
    def wrapper_log_activity():
        print("Before execution of fun")
        val = fun()
        print("After execution of fun")
        return val
    return wrapper_log_activity


@log_activity
def return_greeting(name="John"):
    return f"Hello, {name}"


return_greeting(name="Jimmy")
```



In [198]:
# The wrapper is not expecting a param but one is passed.
# error!
def log_activity(fun):
    def wrapper_log_activity():
        print("Before execution of fun")
        val = fun()
        print("After execution of fun")
        return val
    return wrapper_log_activity

@log_activity
def return_greeting(name="John"):
    return f"Hello, {name}"


return_greeting(name="Jimmy")


TypeError: wrapper_log_activity() got an unexpected keyword argument 'name'

In [200]:
# the following shows that wrapper (the alias) does take any arguments
return_greeting

<function __main__.log_activity.<locals>.wrapper_log_activity()>

In [12]:
# Hardcoding the param works for return_greeting 
# But not for other function we may be interested in logging.
def log_activity(fun):
    def wrapper_log_activity(name="John"):
        print("Before execution of fun")
        val = fun(name)
        print("After execution of fun")
        return val
    return wrapper_log_activity

@log_activity
def return_greeting(name="John"):
    return f"Hello, {name}"


return_greeting(name="Jimmy")


Before execution of fun
After execution of fun


'Hello, Jimmy'

In [202]:
### Debugging Multiple Function

def log_activity(fun):
    def wrapper_log_activity(name="John", ):
        print("Before execution of fun")
        val = fun(name)
        print("After execution of fun")
        return val
    return wrapper_log_activity


@log_activity
def return_greeting(name="John"):
    return f"Hello, {name}"

@log_activity
def return_french_greeting(name="Jonh", tod="morning"):
    if tod == "morning":
        return f"Bonjour, {name}"
    else:
        return f"Bonsoir, {name}"




In [203]:

return_greeting()


Before execution of fun
After execution of fun


'Hello, John'

In [207]:
help(return_greeting)


Help on function wrapper_log_activity in module __main__:

wrapper_log_activity(name='John')



In [18]:
return_french_greeting(name="Julie", tod="evening")

Before execution of fun
After execution of fun


'Bonjour, Julie'

In [206]:
help(return_french_greeting)

Help on function wrapper_log_activity in module __main__:

wrapper_log_activity(name='John')



In [208]:
return_french_greeting.__name__

'wrapper_log_activity'


### Improving Code Readibility

* Note that the help(return_french_greeting) shows the function signature of the alias `wrapper_log_activity`
  * This is counter intuitive/confusing

* Even the method's name has changed
```Python
>>> print(return_french_greeting.__name__)
'wrapper_log_activity'
```
    
* Can be problematic since you may have multiple decorated functions that reference the same wrapper
    
* To fix this, we can use the `wraps` decorator from `functtools`





In [15]:

from functools import wraps

def log_activity(fun):
    @wraps(fun)
    def wrapper_log_activity(name="John"):
        print("Before execution of fun")
        val = fun(name)
        print("After execution of fun")
        return val
    return wrapper_log_activity


@log_activity
def return_greeting(name="John"):
    return f"Hello, {name}"

@log_activity
def return_french_greeting(name="Jonh", tod="morning"):
    if tod == "morning":
        return f"Bonjour, {name}"
    else:
        return f"Bonsoir, {name}"

In [210]:
return_french_greeting

<function __main__.return_french_greeting(name='Jonh', tod='morning')>

In [211]:
help(return_french_greeting)

Help on function return_french_greeting in module __main__:

return_french_greeting(name='Jonh', tod='morning')



In [82]:
return_french_greeting.__name__

'return_french_greeting'

### Passing Arguments to Decorated Funcs - Cont'd

* A versatile approach to handing params is through python's *args and **kwargs

  * `args`: arguments

  * `kwargs`: keyword args.


* `args` and `kwargs` allow you to pass multiple arguments or keyword arguments to a function.

    


In [19]:
def some_function(*args, **kwargs):

    print(f"The args I was passeed are: {args}")
    print(f"The kwargs I was passeed are: {kwargs}")

some_function("A", "Python", name="John", age="33")


The args I was passeed are: ('A', 'Python')
The kwargs I was passeed are: {'name': 'John', 'age': '33'}


### Passing Arguments to Decorated Funcs - Cont'd

* Re-write the wrapper to take as input the `args` and `kwargs`

```python
def log_activity(fun):
    def wrapper_log_activity(*args, **kwargs):
        print("Before execution of fun")
        value = fun(*args, **kwargs)
        print("After execution of fun")
        return value
    return wrapper_log_activity

@log_activity
def return_greeting(name="John"):
    pass

@log_activity
def return_french_greeting(name="Jonh", tod="morning"):
    pass
```


    


In [20]:
def log_activity(fun):
    @wraps(fun)
    def wrapper_log_activity(*args, **kwargs):
        print("Before execution of fun")
        value = fun(*args, **kwargs)
        print("After execution of fun")
        return value
    return wrapper_log_activity

@log_activity
def return_greeting(name="John"):
    return f"Hello, {name}"

@log_activity
def return_french_greeting(name="Jonh", tod="morning"):
    if tod == "morning":
        return f"Bonjour, {name}"
    else:
        return f"Bonsoir, {name}"
    


In [21]:
return_greeting()

Before execution of fun
After execution of fun


'Hello, John'

In [22]:
return_french_greeting(name="Jimmy")

Before execution of fun
After execution of fun


'Bonjour, Jimmy'

In [23]:
return_french_greeting(tod="evening")

Before execution of fun
After execution of fun


'Bonsoir, Jonh'

In [24]:
return_french_greeting(name="Jimmy", tod="evening")

Before execution of fun
After execution of fun


'Bonsoir, Jimmy'

### Important Suggestions About the Returned Values

* The returned value should be only those returned by the decorated function
  * Do not return additional values

* Ex.: I could be tempted to return timestamp, or `True` if action was performed successfully
  * Changes the returned value -- not a good idea 
  
```python
def log_activity(fun):
    @wraps(fun)
    def wrapper_log_activity(*args, **kwargs):
        print("Before execution of fun")
        value = fun(*args, **kwargs)
        print("After execution of fun")
        return value, time.time()
    return wrapper_log_activity
```

In [ ]:
### Decorator Template
```
def some_function(fun):
    @wraps(fun)
    def wrapper_some_function(*args, **kwargs):
        # Do something before execution of fun
        value = fun(*args, **kwargs)
        # Do something after the execution of fun
        return value
    return wrapper_log_activity
```



<b>Hi</b>

### Use Cases: Timing Functions

* Use a decorator to wrap execution of a function
  1. Start timer 
  2. run function 
  3. Stop timer after
  4. Print total time

```python    

def time_me(func):
    @wraps(func)
    def wrapper_time_me(*args, **kwargs):
        start_time = time.time()
        val = func(*args, **kwargs)
        total_time = time.time() - start_time
        print("{func.__name__} completed in {total_time}")
        return val
    return wrapper_time_me
```    

In [221]:
import time

def time_me(func):
    @wraps(func)
    def wrapper_time_me(*args, **kwargs):
        start_time = time.time()
        val = func(*args, **kwargs)
        total_time = time.time() - start_time
        print(f"{func.__name__} completed in {total_time}")
        return val
    return wrapper_time_me

@time_me
def long_computation(sleep_time=3):
    print("Starting to sleep")
    time.sleep(sleep_time)
    print("Woke up")
    

long_computation(4)

Starting to sleep
Woke up
long_computation completed in 4.002820014953613


### Decorators Can be Applied to Class Methods

* The process is the same: decorate the method using the `@`+ the decorating function's name
```python
class SomeClass:
    def __init__(self, x=10):
        self.x = x

    @log_activity
    def say_hi(self, name="Mahdi"):
        print(f"Hi, {name}")
```

In [222]:
class SomeClass:
    def __init__(self, x=10):
        self.x = x

    @log_activity
    def say_hi(self, name="Mahdi"):
        print(f"Hi, {name}")

sc = SomeClass()
sc.say_hi()

Before execution of fun
Hi, Mahdi
After execution of fun


### Decorators with Arguments

* So far, we have allowed the decorated functions to have params but not the decorators.
* How do you customize and change the behavior of a decorator?
  * Decorators should be able to take functions. For example:
  
```python
@log_activity(to="file")
def some_function():
    pass
```       
* `log_activity` could then be used to log to a file, stdout, or even a database if needed




### Decorators with Arguments - Cont'd

* To achieve this, we need another layer of indirection

```python
def Z(param=None):
    def X(Y):
        def wrapper_X(*args, **kwargs):
            # Do Something Before, potentially with param
            value = func(*args, **kwargs)
            # Do Something after, potentially with param
            return value
        return wrapper_X
    return X
```


In [30]:
def Z(param=None):
    def X(func):
        def wrapper_X(*args, **kwargs):
            # Do Something Before, potentially with param
            value = func(*args, **kwargs)
            print(f"The param that was passed is {param}")
            # Do Something after, potentially with param
            return value
        return wrapper_X
    return X


@Z(param="File")
def some_function():
    print("Hi")

some_function()

Hi
The param that was passed is 12


### Use Case: Decorators to Slow Down Non-Blocking Code in Tests

* Given some async code, I need to make sure the code completed before testing the results


```python
x = None
x = some_threaded_function()
assert x == True
```
* The above assert will fail if executed before `some_threaded_function` completes

* Create a decorator that forces the execution to pause -- `sleep()` -- for a given number of seconds



In [31]:
import threading, queue


def some_threaded_function():
    event = threading.Event()
    print("2. In some_threaded_function: Getting ready to do some work that will take four seconds\n")
    event.wait(2)
    print("3. In some_threaded_function: Done doing the work\n")


def run_function_in_thread():
    thread = threading.Thread(target=some_threaded_function)
    thread.start()


    
print("1. Some_threaded_function is starting\n")
run_function_in_thread()   
print("4. function some_threaded_function is done\n")    

1. Some_threaded_function is starting

2. In some_threaded_function: Getting ready to do some work that will take four seconds
4. function some_threaded_function is done


3. In some_threaded_function: Done doing the work



In [33]:
import time

def sleep_factory(interval=2):
    def sleep(func):
        def wrapper_sleep(*args, **kwargs):
            value = func(*args, **kwargs)
            time.sleep(interval)
            return value
        return wrapper_sleep
    return sleep

@sleep_factory(3)
def run_function_in_thread():
    thread = threading.Thread(target=some_threaded_function)
    thread.start()
    
print("1. Some_threaded_function is starting\n")
run_function_in_thread()   
print("4. function some_threaded_function is done\n")    


1. Some_threaded_function is starting

2. In some_threaded_function: Getting ready to do some work that will take four seconds

3. In some_threaded_function: Done doing the work

4. function some_threaded_function is done



### Use Case:  Decorators to Add Attributes on the Fly

* SAGE3 allows users to create Python code and use it in JavaScript front-end

* Python needs a way to introspect objects and return functions that a programmer wants to expose. 

```
class SomeNewFunctionality:
    def __init__(self, a=1, b=2):
        self.a = 1
        self.b = 2
    def expose_this(self):
        pass
    def dont_expose_this(self):
        pass    
```

* We don't want to force the programmer to adopt a specific naming convention.

In [227]:
class SomeNewFunctionality:
    def __init__(self, a=1, b=2):
        self.a = a
        self.b = b

    def expose_this(self):
        print("I do some useful stuff in JavaScript")
    def dont_expose_this(self):
        print("I don't do anything useful in JavaScript")

    

In [228]:
snf = SomeNewFunctionality(3, 5)
snf.expose_this()

I do some useful stuff in JavaScript


In [336]:
def list_obj_methods(some_obj):
    methods  = []
    for method_name in dir(some_obj):
        if callable(getattr(some_obj, method_name)) \
            and not method_name.startswith("__"):
                methods.append(method_name) 

    return methods
list_obj_methods(snf)

['dont_expose_this', 'expose_this']

### Use Case:  Decorators to Add Attributes on the Fly -- Cont'd

* A possible solution can be to use a decorator -- ex. @visible_action -- and let a user decorate function they want to expose


```python
class SomeNewFunctionality:
    def __init__(self, a=1, b=2):
        self.a = 1
        self.b = 2
    @visible_action    
    def expose_this(self):
        pass
    def dont_expose_this(self):
        pass    
```

* Flexible and does not force the programmer to adopt a specific naming convention.



In [337]:
def visible_action(func):
    func.is_visible = True
    def wrapper_visible_action(*args, **kwargs):
            func.is_exposed = True
    return func
    

class SomeNewFunctionality:
    def __init__(self, a=1, b=2):
        self.a = 1
        self.b = 2
        
    @visible_action    
    def expose_this(self):
        print("I do some useful stuff in JavaScript")
        
    def dont_expose_this(self):
        print("I don't do anything useful in JavaScript")


In [339]:
snf = SomeNewFunctionality(3, 5)

list_obj_methods(snf)

['dont_expose_this', 'expose_this']

In [333]:
snf.expose_this()

I do some useful stuff in JavaScript


In [334]:
def list_visible_action(some_obj):
    visible_methods  = []
    for method_name in dir(some_obj):
        if callable(getattr(some_obj, method_name)) \
            and not method_name.startswith("__"):
            method = getattr(some_obj, method_name)
            if hasattr(method, "is_visible"):
                visible_methods.append(method_name) 

    return visible_methods
list_visible_action(snf)

['expose_this']